# Configure a Label Switched Network
###### <sup>Developed by: Sarah Samuel (sasamuel@cisco.com). </sup>

This notebook shows you the code required to bring up a small MPLS network with predefined configurations, using the pyvxr API. These functions are then used in the ospf_mpls.py library which we use to bring this network up again in other notebooks.

Open Shortest Path First (OSPF) is an Interior Gateway Protocol (IGP) developed by the OSPF working group of the Internet Engineering Task Force (IETF). OSPF is the IGP of choice because it scales to large networks. It uses areas to partition the network into more manageable sizes and to introduce hierarchy in the network. A router is attached to one or more areas in a network. All of the networking devices in an area maintain the same complete database information about the link states in their area only. 

MPLS, with its label switching capabilities, eliminates the need for an IP route look-up and creates a virtual circuit (VC) switching function, allowing service-provider networks the same performance on their IP-based network services as with those delivered over traditional networks such as Frame Relay or ATM.

Label Distribution Protocol (LDP) performs label distribution in MPLS environments. LDP provides the following capabilities:

* LDP performs hop-by-hop or dynamic path setup; it does not provide end-to-end switching services.

* LDP assigns labels to routes using the underlying Interior Gateway Protocols (IGP) routing protocols.

* LDP provides constraint-based routing using LDP extensions for traffic engineering.

* Finally, LDP is deployed in the core of the network and is one of the key protocols used in MPLS-based Layer 2 and Layer 3 virtual private networks (VPNs).

Traditionally, Service Provider (SP) networks consisted of Provider (P) routers at the core of the SP network and Provider Edge (PE) routers at the edge of the SP network. This notebook brings up a topology with OSPF as the IGP and a MPLS LDP core, with predefined configs. It consists of 2 core routers and 2 edge routers. This topology can be used as a base for further experimenting with other the features used in the service provider network.

![SP Diagram](ospf-mpls.png)


In this notebook you will be taken through the basic steps to set up the simulated topology in a notebook:

* [Set up emulator](#step1)
* [Add the base configuration](#step2)
* [Design the virtual router topology](#step3)
* [Deploy the configurations](#step4)

## <a name="step1"></a>Set up Emulator

>First, you'll have to set up the emulator to build your network. The following code cells sets up the python enviroment and the emulator object.

In [1]:
from pathlib import Path
from pyvxr.vxr import Vxr
import sys
import os
import logging
import shutil
import getpass
import string
import random
print(sys.version)
logging.basicConfig(level=logging.INFO)
sys.path.append("../../../")
from image_version import *

# Setting up scratch space for the simulation
sim_dir = '/nobackup/' + getpass.getuser() + '/pyvxr/' + ''.join(random.choices(string.ascii_lowercase + string.digits, k=10))

print(sys.version)
logging.basicConfig(level=logging.INFO)
sim = Vxr()
sim.no_image_copy=True

INFO:pyvxr.vxr:v1.1.0 2021-04-23 05:42 output_dir:vxr.out
INFO:pyvxr.vxr:b10e8f6b8c14:/home/vxr/notebooks/Getting-Started/Setting-Up-Basic-Network/Ospf-Mpls


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]


In [2]:
# Nice little python trick to peek at the API of the sim object.
dir(sim)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allocate_slurm',
 '_analyze_logs',
 '_check_sim_config_yaml_file',
 '_clean_sim',
 '_connect_distributed_hubs',
 '_create_fresh_dir',
 '_create_port_yaml_file',
 '_create_vxrsim_cfg_file',
 '_download_dot_server_files',
 '_fix_vxr_release_string',
 '_get_pyvxr_flag',
 '_get_vxr_ports_file',
 '_get_vxr_ports_files',
 '_get_vxr_rel',
 '_get_vxr_ver',
 '_get_vxr_version',
 '_logs_from_host',
 '_parse_routers',
 '_portVectorFileName',
 '_post_sim_launch',
 '_restart_cards',
 '_restart_common',
 '_restart_sim',
 '_run_sim_check',
 '_send_email',
 '_setup_local_ssh_tunnels',
 '_setup_ssh_tunnels',
 '_start_common'

## <a name="step2"></a>Add the Base Configurations

>This sets up the configurations for a 4 router topology (2 P and 2 PE routers) for our lab and a Linux server to use on the same LAN, with SSH to all of the routers.

In [3]:
p1_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh

hostname P1
line console
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
line default
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
 
ipv4 unnumbered mpls traffic-eng Loopback0
!
interface Loopback0
 ipv4 address 10.0.0.5 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 description Connected_to_P3
 mtu 9216
 ipv4 address 10.5.7.1 255.255.255.0
 no shutdown
!
interface FourHundredGigE0/0/0/1
 description Connected_to_PE1
 mtu 9216
 ipv4 address 10.20.7.1 255.255.255.0
 no shutdown
 !
router ospf 10
 router-id 10.0.0.5
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
  !
 !
!
mpls oam
!
mpls ldp
 router-id 10.0.0.5
 interface FourHundredGigE0/0/0/0
 interface FourHundredGigE0/0/0/1
 !
!
multicast-routing
 address-family ipv4
  interface all enable
 !
 """

In [4]:
p3_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh

hostname P3
!
ipv4 unnumbered mpls traffic-eng Loopback0
!
interface Loopback0
 ipv4 address 10.0.0.7 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 description Connected_to_P1
 mtu 9216
 ipv4 address 10.5.7.2 255.255.255.0
 no shutdown
!
interface FourHundredGigE0/0/0/1
 description Connected_to_PE3
 mtu 9216
 ipv4 address 10.21.7.1 255.255.255.0
 no shutdown
!
router ospf 10
 router-id 10.0.0.7
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
  !
 !
!
mpls oam
!
mpls ldp
 router-id 10.0.0.7
 interface FourHundredGigE0/0/0/0
 interface FourHundredGigE0/0/0/1
 !
!
multicast-routing
 address-family ipv4
  interface all enable
 !
!
"""

In [5]:
pe1_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh

hostname PE1
line console
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
line default
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
 
ipv4 unnumbered mpls traffic-eng Loopback0
!
interface Loopback0
 ipv4 address 10.0.0.3 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 no shutdown
!
interface FourHundredGigE0/0/0/1
 description Connected_to_P1
 mtu 9216
 ipv4 address 10.20.7.2 255.255.255.0
 no shutdown
 !
router ospf 10
 router-id 10.0.0.3
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
  !
 !
!
mpls oam
!
mpls ldp
 router-id 10.0.0.3
 interface FourHundredGigE0/0/0/0
 interface FourHundredGigE0/0/0/1
 !
!
multicast-routing
 address-family ipv4
  interface all enable
 !
 """

In [6]:
pe3_config_str = """
ssh server v2
ssh server netconf
netconf-yang agent ssh

hostname PE3
line console
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
line default
 exec-timeout 0 0
 absolute-timeout 0
 session-timeout 0
!
 
ipv4 unnumbered mpls traffic-eng Loopback0
!
interface Loopback0
 ipv4 address 10.0.0.9 255.255.255.255
!
interface FourHundredGigE0/0/0/0
 no shutdown
!
interface FourHundredGigE0/0/0/1
 description Connected_to_P3
 mtu 9216
 ipv4 address 10.21.7.2 255.255.255.0
 no shutdown
 !
router ospf 10
 router-id 10.0.0.9
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
  !
 !
!
mpls oam
!
mpls ldp
 router-id 10.0.0.9
 interface FourHundredGigE0/0/0/0
 interface FourHundredGigE0/0/0/1
 !
!
multicast-routing
 address-family ipv4
  interface all enable
 !
 """

## <a name="step3"></a>Design the Virtual Router Topology

>This sets up the 4 router topology (spitfire_f).

In [8]:
cfg = { 'simulation':
         {'skip_auto_bringup': False, 
          'sim_dir': sim_dir, 
          'sim_host': 'localhost',
          'sim_rel': '/opt/cisco/vxr2/latest',
          'pyvxr_flags': {'port_file_timeout': 1200 }
         },
        'devices':
        {'rp1': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'linecard_types': ['8201-sys'], 
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1'],
                'xr_config' : p1_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
                },
         'rp3': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'linecard_types': ['8201-sys'], 
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1'],
                'xr_config' : p3_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
                },
         'rpe1': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'linecard_types': ['8201-sys'],  
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1'],
                'xr_config' : pe1_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
                },
         'rpe3': {'platform':'spitfire_f-baked',
                'xr_port_redir': [22, 830],
                'linecard_types': ['8201-sys'],  
                'data_ports': ['FourH0/0/0/0', 'FourH0/0/0/1'],
                'xr_config' : pe3_config_str,
                'image': sim_image_global,
                'vxr_sim_config': {
                     'shelf': {
                       'ConfigOvxr': ConfigOvxr_global,
                       'ConfigEnableNgdp': ConfigEnableNgdp_global,
                       'ConfigS1SdkVer': ConfigS1SdkVer_global,
                       'ConfigS1NpsuiteVer': ConfigS1NpsuiteVer_global
                     }
                  }
                },
        },
       'connections':
           {'hubs':
               {'hub570':['rp1.FourH0/0/0/0', 'rp3.FourH0/0/0/0'],
               'hub571':['rpe1.FourH0/0/0/1', 'rp1.FourH0/0/0/1'],
               'hub572':['rpe3.FourH0/0/0/1', 'rp3.FourH0/0/0/1']}
           },
      }



## <a name="step4"></a>Deploy the Configurations

>Now we initialise the emulator with the previously defined configurations and topology (ie the **cfg** variable), then bring it up. The bring up can be slow, 10 minutes +.

In [9]:
sim.clean()
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")
try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/y90isrs7t2 sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:rp1, rp3, rpe1, rpe3
INFO:pyvxr.console:rpe1:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe1.log)
INFO:pyvxr.console:rp1:wait for XR login prompt (console output captured in vxr.out/logs/console.rp1.log)
INFO:pyvxr.console:rp3:wait for XR login prompt (console output captured in vxr.out/logs/console.rp3.log)
INFO:pyvxr.console:rpe3:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe3.log)
INFO:pyvxr.console:rp1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:rp1:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:rp1:login successful
INFO:pyvxr.bringup:rp1

Sim status:  {'localhost': 'running'}


In [10]:
# Defining two little helpers to obtain the telnet and ssh ip addresses and ports to the routers.

def get_telnet_cmd(sim, router):
    """Get a telnet command to a router in a VXR simulation.
    Keyword arguments:
    sim -- an instance of the Vxr object
    router -- the router name in the simulation
    """
    console_ports = sim.ports()
    return "telnet " + str(console_ports[router]['HostAgent']) + ' ' + str(console_ports[router]['serial0'])

def get_ssh_cmd(sim, device, is_server=False):
    """Get a telnet command to a router in a VXR simulation
    Keyword arguments:
    sim -- an instance of the Vxr object
    router -- the router name in the simulation
    """
    console_ports = sim.ports()
    if (is_server):
        return "ssh root@" + str(console_ports[device]['HostAgent']) + ' -p' + str(console_ports[device]['xr_redir22'])
    else:
        return "ssh cisco@" + str(console_ports[device]['HostAgent']) + ' -p' + str(console_ports[device]['xr_redir22'])

>At this point, console access to the router is available. Optionally, you can access the simulated router consoles directly from your laptop  through telnet or ssh.

In [11]:
print('Consoles can be reached by:')
print(get_telnet_cmd(sim, 'rp1'), '\n', get_telnet_cmd(sim, 'rp3'), '\n', get_telnet_cmd(sim, 'rpe1'),  '\n', get_telnet_cmd(sim, 'rpe3'))
print('or better:')
print(get_ssh_cmd(sim, 'rp1'), '\n', get_ssh_cmd(sim, 'rp3'), '\n', get_ssh_cmd(sim, 'rpe1'),  '\n', get_ssh_cmd(sim, 'rpe3'))
print('The password is cisco123')

Consoles can be reached by:
telnet 172.17.0.2 39210 
 telnet 172.17.0.2 41988 
 telnet 172.17.0.2 43910 
 telnet 172.17.0.2 37439
or better:
ssh cisco@172.17.0.2 -p63075 
 ssh cisco@172.17.0.2 -p64956 
 ssh cisco@172.17.0.2 -p64689 
 ssh cisco@172.17.0.2 -p65443
The password is cisco123


> You can either open your terminal and directly access the consoles of the routers or you can just continue playing the next steps and access the router consoles from this notebook.

### Verify the Features

In [12]:
import telnetlib
console_ports = sim.ports()
loginpe1 = telnetlib.Telnet(str(console_ports['rpe1']['HostAgent']) , str(console_ports['rpe1']['serial0']))
loginp1 = telnetlib.Telnet(str(console_ports['rp1']['HostAgent']) , str(console_ports['rp1']['serial0']))
loginp3 = telnetlib.Telnet(str(console_ports['rp3']['HostAgent']) , str(console_ports['rp3']['serial0']))
loginpe3 = telnetlib.Telnet(str(console_ports['rpe3']['HostAgent']) , str(console_ports['rpe3']['serial0']))

> Keep replaying this cell until the **State** of OSPF is **FULL/DR** 

In [19]:
loginpe1.write(('''
show ip ospf neighbor
''').encode('ascii'))
line = loginpe1.read_until(b'/r/n',4)
print(line.decode())


RP/0/RP0/CPU0:PE1#show ip ospf neighbor
Fri Apr 23 05:50:46.060 UTC

* Indicates MADJ interface
# Indicates Neighbor awaiting BFD session up

Neighbors for OSPF 10

Neighbor ID     Pri   State           Dead Time   Address         Interface
10.0.0.5        1     FULL/DR         00:00:34    10.20.7.1       FourHundredGigE0/0/0/1
    Neighbor is up for 00:01:50

Total neighbor count: 1
RP/0/RP0/CPU0:PE1#


In [20]:
loginpe1.write(('''
show mpls ldp neighbor
''').encode('ascii'))
line = loginpe1.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:PE1#show mpls ldp neighbor
Fri Apr 23 05:50:52.147 UTC

Peer LDP Identifier: 10.0.0.5:0
  TCP connection: 10.0.0.5:21199 - 10.0.0.3:646
  Graceful Restart: No
  Session Holdtime: 180 sec
  State: Oper; Msgs sent/rcvd: 12/12; Downstream-Unsolicited
  Up time: 00:01:25
  LDP Discovery Sources:
    IPv4: (1)
      FourHundredGigE0/0/0/1
    IPv6: (0)
  Addresses bound to this peer:
    IPv4: (4)
      10.0.0.5       10.5.7.1       10.20.7.1      192.168.254.11  
    IPv6: (0)

RP/0/RP0/CPU0:PE1#


In [21]:
loginp1.write(('''
show ip ospf neighbor
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:P1#show ip ospf neighbor
Fri Apr 23 05:50:54.042 UTC

* Indicates MADJ interface
# Indicates Neighbor awaiting BFD session up

Neighbors for OSPF 10

Neighbor ID     Pri   State           Dead Time   Address         Interface
10.0.0.7        1     FULL/DR         00:00:32    10.5.7.2        FourHundredGigE0/0/0/0
    Neighbor is up for 00:02:01
10.0.0.3        1     FULL/BDR        00:00:32    10.20.7.2       FourHundredGigE0/0/0/1
    Neighbor is up for 00:02:00

Total neighbor count: 2
RP/0/RP0/CPU0:P1#


In [22]:
loginp1.write(('''
show mpls ldp neighbor
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print(line.decode())


RP/0/RP0/CPU0:P1#show mpls ldp neighbor
Fri Apr 23 05:50:56.050 UTC

Peer LDP Identifier: 10.0.0.3:0
  TCP connection: 10.0.0.3:646 - 10.0.0.5:21199
  Graceful Restart: No
  Session Holdtime: 180 sec
  State: Oper; Msgs sent/rcvd: 12/12; Downstream-Unsolicited
  Up time: 00:01:29
  LDP Discovery Sources:
    IPv4: (1)
      FourHundredGigE0/0/0/1
    IPv6: (0)
  Addresses bound to this peer:
    IPv4: (3)
      10.0.0.3       10.20.7.2      192.168.254.66  
    IPv6: (0)

Peer LDP Identifier: 10.0.0.7:0
  TCP connection: 10.0.0.7:23653 - 10.0.0.5:646
  Graceful Restart: No
  Session Holdtime: 180 sec
  State: Oper; Msgs sent/rcvd: 12/12; Downstream-Unsolicited
  Up time: 00:01:25
  LDP Discovery Sources:
    IPv4: (1)
      FourHundredGigE0/0/0/0
    IPv6: (0)
  Addresses bound to this peer:
    IPv4: (4)
      10.0.0.7       10.5.7.2       10.21.7.1      192.168.254.142  
    IPv6: (0)

RP/0/RP0/CPU0:P1#


>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [23]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/y90isrs7t2)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [24]:
sim.clean()

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/y90isrs7t2)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


> Clean up the sim scratch-space - delete the sim directory 

In [25]:
shutil.rmtree(sim_dir)